# naver 아티클 크롤링

In [31]:
import requests

In [6]:
from selenium import webdriver

In [2]:
category = {
    100: "정치",
    101: "경제",
    102: "사회",
    103: "생활/문화",
    104: "세계",
    105: "IT/과학"
}

In [4]:
category, date, page = 100, 20210101, 1

url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1={}&date={}&page={}".format(category, date, page)

In [7]:
driver = webdriver.Chrome()
driver.get(url)

In [34]:
articles = driver.find_elements_by_css_selector(".list_body.newsflash_body > ul > li")
len(articles)

20

In [35]:
articles[0].text

'문 대통령, 공군지휘통제기 지휘비행...영공 방어 점검\n문재인 대통령은 새해 첫날 공군지휘통제기에 탑승해 지휘비행을 하고 영공 방어 태세를 점검했습니다. 문 대통령은 오늘 오전 공군 …\nYTN 1분전'

In [36]:
title = articles[0].find_element_by_css_selector('dt:not(.photo) > a ').text
link = articles[0].find_element_by_css_selector('dt:not(.photo) > a').get_attribute("href")
title, link

('문 대통령, 공군지휘통제기 지휘비행...영공 방어 점검',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=052&aid=0001533109')

In [39]:
response = requests.get(link)
dom = BeautifulSoup(response.content, "html.parser")

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [40]:
content = dom.select_one("#articleBodyContents").text.replace("\n","").split("{}")[1].strip()
content

NameError: name 'dom' is not defined

In [41]:
def get_articles(catergory, date, page):
    url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1={}&date={}&page={}".format(category, date, page)
    driver = webdriver.Chrome()
    driver.get(url)
    articles = driver.find_elements_by_css_selector(".list_body.newsflash_body > ul > li")
    datas = []
    for article in articles:
        title = article.find_element_by_css_selector('dt:not(.photo) > a ').text
        link = article.find_element_by_css_selector('dt:not(.photo) > a').get_attribute("href")
        response = requests.get(link)
        dom = BeautifulSoup(response.content, "html.parser")
        content = dom.select_one("#articleBodyContents").text.replace("\n","").split("{}")[1].strip()
        datas.append({
            "title" : title,
            "link" : link,
            "content" : content,
            "category" : category,
        })
    driver.quit()
    
    return pd.DataFrame(datas)

In [ ]:
dfs = []

for category in categories:
    print(categories[category], end= " ")    
    
    for page in range(1, 6):
        print(page, end=" ")
        df = get_articles(category, date, page)
        dfs.append(df)

articles_df = pd.concat(dfs, ignore_index=True)
articles_df.tail()

In [ ]:
articles_df.to_csv('article.csv', index=False, encoding="utf-8-sig")

## 데이터 가져오기

In [42]:
df = pd.read_csv('article.csv')

In [43]:
df.tail()

,category,content,link,title
595,105,"[디지털데일리 채수웅기자] 과학기술정보통신부(장관 최기영)는 21일 KT, SK텔레...",https://news.naver.com/main/read.nhn?mode=LSD&...,"과기정통부-통신사, 재난방지 대책 추진현황 점검"
596,105,"게임 기획 단계부터 타격감 위주로 설계(지디넷코리아=김한준 기자)""슬랩잇은 기획 ...",https://news.naver.com/main/read.nhn?mode=LSD&...,STAC 2019 게임 수상작 '슬랩잇' 고등학생 퀄리티 넘어섰다
597,105,"과기정통부, 22일자로 IBS 신임 원장 임명[아이뉴스24 최상국 기자] 기초과학연...",https://news.naver.com/main/read.nhn?mode=LSD&...,"노도영 IBS 신임 원장 임명, ""IBS 개혁 · 중이온가속기 성공"" 임무"
598,105,병역 대체복무제도 개선 확정…중소·중견기업에 배정 늘려(세종=연합뉴스) 신선미 기자...,https://news.naver.com/main/read.nhn?mode=LSD&...,2025년 전문연구요원 석사 300명 감축…박사는 1천명 유지(종합)
599,105,엔지니어들이 SOC실증연구센터에서 스마트 건설 방식으로 공사를 진행하고 있다. 제...,https://news.naver.com/main/read.nhn?mode=LSD&...,"SKT, ‘5G+건설·기계’ 드론 띄워 측량, 굴삭기로 정밀공사 ‘착착’"
